In [45]:
import numpy as np

MetaID = 0
MetaDATA = 1
MetaLABEL = 2
MetaSTART = 3
MetaDURATION = 4

print("Loading data...")
metadata = np.loadtxt('/mnt/ExtDrive/sensor data/HT_Sensor_metadata.dat', skiprows=1, dtype=str)
dataset = np.loadtxt('/mnt/ExtDrive/sensor data/HT_Sensor_dataset.dat', skiprows=1)
print("Preprocessing data...")

# data[0] - Linear, no background reduced
# data[1] - Linear, background reduced
# data[2] - Quadratic, no background reduced
# data[3] - Quadratic, background reduced
banana_data = [[], [], [], [], [], []]
wine_data = [[], [], [], [], [], []]
background_data = [[], [], [], [], [], []]

def sqr_features(features, length):
    features_sqr = np.resize(features, length)
    idx = len(features)
    for i in range(0, len(features)):
        for j in range(0, i + 1):
            features_sqr[idx] = features[i] * features[j]
            idx += 1
    return features_sqr

def sqr_features_str(features):
    num_features = len(features)
    for i in range(0, num_features):
        for j in range(0, i + 1):
            features.append(features[i] + "*" + features[j])

feature_names1 = ["time", "R1", "R2", "R3", "R4", "R5", "R6", "R7", "R8", "temp", "h"]
feature_names2 = feature_names1.copy()
for i in range(0, len(feature_names2)):
    feature_names2.append(feature_names2[i] + "'")
sqr_features_str(feature_names1)
sqr_features_str(feature_names2)

segment_len = 1 / 60 # one minute increments
next_segment = 0
cur_ind = None
cur_segment_data = []
cur_background_data = None
p_data = None
for entry in dataset:
    if cur_ind is None or int(entry[0]) != int(cur_ind[MetaID]):
        cur_ind = metadata[int(entry[0])]
        cur_background_data = None
        cur_segment_data = []
        next_segment = 0
        
        if cur_ind[MetaLABEL] == 'banana':
            p_data = banana_data
        elif cur_ind[MetaLABEL] == 'wine':
            p_data = wine_data
        else:
            p_data = background_data
        
    cur_segment_data.append(entry[1:])
    if entry[1] < next_segment or entry[1] > float(cur_ind[MetaDURATION]):
        continue
    
    if cur_background_data is None:
        cur_background_data = np.mean(np.array(cur_segment_data), axis=0)
    else:
        p_data[0].append(entry[1:])
        p_data[1].append(np.concatenate((entry[1:], entry[1:] - cur_background_data)))
        p_data[2].append(sqr_features(p_data[0][len(p_data[0])-1], len(feature_names1)))
        p_data[3].append(sqr_features(p_data[1][len(p_data[1])-1], len(feature_names2)))
        
        # Comment out for PCA
        #for i in range(0, 4):
        #    p_data[i][len(p_data[i]) - 1] = np.concatenate((p_data[i][len(p_data[i]) - 1], [1]))
    
    cur_segment_data = []
    next_segment += segment_len

print("Done")
print(len(banana_data[0]))
print(len(wine_data[0]))
print(len(background_data[0]))

#   0: id
#   1: time
# 2-9: R1-R8
#  10: temperature
#  11: humidity

Loading data...
Preprocessing data...
77
275
Done
1375
1883
1130


In [51]:
print(len(banana_data[0]), len(banana_data[0][0]))

display_thresholds = [0.4, 0.3, 0.154, 0.095]
keep_threshold = 1
for i in range(0, 4):
    print("PCA", i)
    
    all_data = np.concatenate((banana_data[i], wine_data[i], background_data[i]))
    all_data_z = all_data - np.mean(all_data, axis=0)
    all_data_z /= np.std(all_data_z, axis=0)
    U, s, VT = np.linalg.svd(all_data_z, full_matrices=False)
    
    #print(s[:20])
    p = VT[0,:]
    for j in range(0, len(p)):
        if np.abs(p[j]) > display_thresholds[i]:
            if i == 0 or i == 2:
                print(feature_names1[j], p[j])
            else:
                print(feature_names2[j], p[j])
    
    if i >= 2:
        for j in range(0, len(s)):
            if s[j] < keep_threshold:
                principleComponents = np.transpose(VT[0:j,:])
                print(principleComponents.shape)
                banana_data[i+2] = np.array(banana_data[i]) @ principleComponents
                wine_data[i+2] = np.array(wine_data[i]) @ principleComponents
                background_data[i+2] = np.array(background_data[i]) @ principleComponents
                break
        
        #for j in range(0, len(banana_data[i])):
        #    banana_data[i+2].append()

1375 11
PCA 0
R1 -0.4330109137197943
R2 -0.4540422998551683
R3 -0.4658909928586077
R4 -0.46483945865896203
PCA 1
R3 -0.3024432220608651
R1' -0.31365514452433246
R2' -0.3105910064569595
R3' -0.32515044168217655
R4' -0.3176143120809689
PCA 2
R3 -0.15494089104285083
R3*R1 -0.15639163664026828
R3*R2 -0.15651380381162994
R3*R3 -0.15430894813055118
R4*R1 -0.15445990466703718
R4*R3 -0.15641525318749017
temp*R3 -0.15456289841140594
h*R3 -0.15481520082016745
(77, 55)
PCA 3
R1'*R7 -0.09855646010003023
R1'*R8 -0.09735558195919089
R3'*R7 -0.09745098869150924
R3'*R8 -0.09837827288078016
R4'*R8 -0.0953503110956183
(275, 148)


In [56]:

def ridge_regression(test_data, background_data):
    # Combine positive samples with background samples.
    Xb = np.concatenate((test_data, background_data))
    yb = np.concatenate((np.full(len(test_data), 1), np.full(len(background_data), -1)))

    # Shuffle so that the background samples aren't all in the back.
    np.random.seed(0)
    np.random.shuffle(Xb)
    np.random.seed(0)
    np.random.shuffle(yb)

    num_samples, num_features = Xb.shape
    NUM_PARTS = 8

    # Set up partitions
    setIndices = [[0, int(num_samples / NUM_PARTS)]]
    for i in range(1, NUM_PARTS):
        prev = setIndices[i-1][1]
        setIndices.append([prev, int(prev+num_samples/NUM_PARTS)])
    setIndices[NUM_PARTS-1][1] = num_samples

    holdOuts = [[0, 1]]
    for i in range(1, NUM_PARTS - 1):
        prev = holdOuts[i-1][1]
        holdOuts.append([prev, prev+1])

    lam_vals = [0.1, 0.2, 0.4, 1, 2, 4, 10, 20, 40, 100, 200, 400]

    errorRate = 0
    lambdas = 0
    for j in range(0, len(holdOuts)):
        v1Ind = np.arange(setIndices[holdOuts[j][0]][0], setIndices[holdOuts[j][0]][1])
        v2Ind = np.arange(setIndices[holdOuts[j][1]][0], setIndices[holdOuts[j][1]][1])
        trnInd = list(set(range(len(yb))) - set(v1Ind) - set(v2Ind))

        Xt = Xb[trnInd,:]
        yt = yb[trnInd]

        Xv1 = Xb[v1Ind,:]
        yv1 = yb[v1Ind]
        Xv2 = Xb[v2Ind,:]
        yv2 = yb[v2Ind]

        minErr = float("inf")
        minIdx = 0
        minW = []
        for i in range(0, len(lam_vals)):
            w = np.linalg.inv(Xt.T@Xt + lam_vals[i] * np.identity(num_features))@Xt.T@yt

            error = np.abs(np.sign(Xv1@w) - yv1).sum()
            if error <= minErr:
                minErr = error
                minIdx = i
                minW = w
        #print("Min lam:", lam_vals[minIdx])
        #print("Min err:", minErr)
        lambdas += lam_vals[minIdx]
        errorRate += np.abs(np.sign(Xv2@minW) - yv2.flatten()).sum() / (2 * len(yv2))

    errorRate /= len(holdOuts)
    return errorRate, lambdas / len(lam_vals)

print("Without background-removed features:")
print("  Linear:")
print("    Banana:", ridge_regression(banana_data[0], background_data[0]))
print("    Wine:", ridge_regression(wine_data[0], background_data[0]))
print("  Quadratic:")
print("    Banana:", ridge_regression(banana_data[2], background_data[2]))
print("    Wine:", ridge_regression(wine_data[2], background_data[2]))
print("  Quadratic+PCA:")
print("    Banana:", ridge_regression(banana_data[4], background_data[4]))
print("    Wine:", ridge_regression(wine_data[4], background_data[4]))
print("With background-removed features:")
print("  Linear:")
print("    Banana:", ridge_regression(banana_data[1], background_data[1]))
print("    Wine:", ridge_regression(wine_data[1], background_data[1]))
print("  Quadratic:")
print("    Banana:", ridge_regression(banana_data[3], background_data[3]))
print("    Wine:", ridge_regression(wine_data[3], background_data[3]))
print("  Quadratic+PCA:")
print("    Banana:", ridge_regression(banana_data[5], background_data[5]))
print("    Wine:", ridge_regression(wine_data[5], background_data[5]))

Without background-removed features:
  Linear:
    Banana: (0.13731478224467786, 32.03333333333333)
    Wine: (0.1016541815251817, 114.16666666666667)
  Quadratic:
    Banana: (0.022810455046266283, 216.66666666666666)
    Wine: (0.0436032596989206, 91.7)
  Quadratic+PCA:
    Banana: (0.022811908589568792, 183.33333333333334)
    Wine: (0.04437311027610911, 43.333333333333336)
With background-removed features:
  Linear:
    Banana: (0.06068252579312589, 3.3333333333333335)
    Wine: (0.08079242754230188, 134.16666666666666)
  Quadratic:
    Banana: (0.0041048062862840745, 2.9166666666666665)
    Wine: (0.008728629665972603, 0.3916666666666666)
  Quadratic+PCA:
    Banana: (0.01551076058106847, 56.18333333333334)
    Wine: (0.026915850944163894, 2.1999999999999997)
